# Preparing data

In [1]:
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new nsteps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."


In [4]:
corpus = data.split('\n')
print("Length of peom : ", len(corpus))

Length of peom :  64


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
print("Total number of words : ", len(tokenizer.word_index))

Total number of words :  262


## Preparing Training data

In [39]:
training_data = []
max_length = 0
for line in corpus:
    line_sequence = tokenizer.texts_to_sequences([line])[0]
    if len(line_sequence) > max_length:
        max_length = len(line_sequence)
    for index in range(1, len(line_sequence)):
        training_data.append(line_sequence[:index+1])

In [40]:
print("Train data size", len(training_data))

Train data size 453


In [43]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_training_data = pad_sequences(training_data, maxlen=max_length, padding='pre')

## Saperating labels and sentences

In [47]:
padded_training_sent, padded_training_label = padded_training_data[:,:-1], padded_training_data[:,-1]

In [48]:
import tensorflow as tf

padded_training_label_one_hot = tf.one_hot(padded_training_label, len(tokenizer.word_index))

In [69]:
print(padded_training_label[2])
print(padded_training_label_one_hot[2].shape[0])

8
262


# Model

In [72]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(padded_training_label_one_hot[2].shape[0], 64, input_length=max_length-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(padded_training_label_one_hot[2].shape[0], activation='softmax')
])
model.compile(loss='categorical_crossentropy', metrics=["accuracy"], optimizer="adam")

In [71]:
history = model.fit(padded_training_sent, padded_training_label_one_hot, epochs=500, verbose=1)

Epoch 1/500
 7/15 [=============>................] - ETA: 0s - loss: 5.5490 - accuracy: 0.0000e+00 

InvalidArgumentError:  indices[3,7] = 262 is not in [0, 262)
	 [[node sequential_5/embedding_5/embedding_lookup (defined at <ipython-input-71-815596622bf5>:1) ]] [Op:__inference_train_function_18161]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_5/embedding_5/embedding_lookup:
 sequential_5/embedding_5/embedding_lookup/15981 (defined at /Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/contextlib.py:113)

Function call stack:
train_function
